In [314]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.linear_model import LogisticRegression

pd.options.display.max_columns = 999
pd.set_option('display.max_rows', 100)

df=pd.read_csv("merge_outer_x_y.csv")

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,6,10,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Add X16,17,18 to df

In [315]:
df2=pd.read_csv("prices_X16_X17_X18.csv")
df2=df2[["pSP_CUSIP","pSP_DATE","30D","60D","90D"]]
df=df.merge(df2,how="left",left_on=["iCUSIP","iPERIOD_END"], right_on=["pSP_CUSIP","pSP_DATE"])
df.drop(columns=["pSP_CUSIP","pSP_DATE"],inplace=True)

df.dropna(inplace=True)

In [316]:
df.head()

,iCUSIP,iPERIOD_END,x1,x4,x5,x2,x3,x7,good_subfund_ornot,x6,x8_mean,x9,x10,x11,x12,good_vssub_ornot,x13,x14,x15,x20,x21,period_now,period_next,return,30D,60D,90D
3917,00206R102,2013-12-31,66.0,0.087343,0.154217,Communications,MidCap Value,0.007457,1.0,0.008125,-0.01304460231964286,0.593877,0.076923,0.051945,0.069261,1.0,0.036052,-0.026331,0.865647,118696.123853,1061.990826,2014-02-14,2014-05-15,0.101659,5.329071e-15,0.278352,0.201912
3918,00206R102,2013-12-31,66.0,0.087343,0.154217,Communications,MidCap Value,0.007457,1.0,0.008125,-0.01304460231964286,0.593877,0.076923,0.051945,0.069261,1.0,0.036052,-0.045538,0.865647,118696.123853,1061.990826,2014-02-14,2014-05-15,0.101659,5.329071e-15,0.278352,0.201912
3919,00206R102,2013-12-31,66.0,0.087343,0.154217,Communications,MidCap Value,0.007457,1.0,0.008125,-0.01304460231964286,0.593877,0.076923,0.051945,0.069261,1.0,0.036052,-0.054152,0.865647,118696.123853,1061.990826,2014-02-14,2014-05-15,0.101659,5.329071e-15,0.278352,0.201912
3920,00206R102,2013-12-31,66.0,0.087343,0.154217,Communications,MidCap Value,0.007457,1.0,0.008125,-0.01304460231964286,0.593877,0.076923,0.051945,0.069261,1.0,0.036052,-0.052476,0.865647,118696.123853,1061.990826,2014-02-14,2014-05-15,0.101659,5.329071e-15,0.278352,0.201912
3921,00206R102,2013-12-31,66.0,0.087343,0.154217,Communications,MidCap Value,0.007457,1.0,0.008125,-0.01304460231964286,0.593877,0.076923,0.051945,0.069261,1.0,0.036052,-0.044721,0.865647,118696.123853,1061.990826,2014-02-14,2014-05-15,0.101659,5.329071e-15,0.278352,0.201912


In [317]:
# 把非数字类型类别数字化
label_encoder =LabelEncoder()
df['x2']= label_encoder.fit_transform(df['x2'])
df['x3']= label_encoder.fit_transform(df['x3'])
# Logistic Regression要求把return转化为是和否两种类型 
df["return"]=np.where(df["return"]>0, 1,0)

In [318]:
X=df.drop(columns=["return","iCUSIP","iPERIOD_END","period_now","period_next"])
y= df["return"]

In [319]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model = LogisticRegression()
clf=model.fit(X_train, y_train)

In [320]:
pd.DataFrame(clf.coef_.T,X.columns,columns=["Coefficient"])

,Coefficient
x1,0.004505
x4,0.000015
x5,0.000010
x2,0.000662
x3,0.000124
x7,0.000002
good_subfund_ornot,0.000092
x6,0.000005
x8_mean,0.000002
x9,0.000049


In [321]:
#测试

clf.score(X_test, y_test)

0.665929203539823

In [322]:
from sklearn.metrics import confusion_matrix

clf_predicted = clf.predict(X_test)
confusion = confusion_matrix(y_test, clf_predicted)
print('混淆矩阵\n', confusion)

混淆矩阵
 [[ 176  703]
 [ 203 1630]]
